In [65]:
using Random
using Distributions
using StatsBase
using FLoops

include("main.jl")

In [70]:
"""
DATA CONFIG
"""
alpha = 0.05

# data
B  = 1    # num. coverage probabilities per boxplot
S  = 1800   # num. samples per coverage probability
nx = 8      # size of group 1
ny = 8      # size of group 2
dtype = Float32
seed = 123

"""
POPULATION SETTINGS
"""

Random.seed!(seed)
distrTypeX = LogNormal{dtype}
muX = dtype.(round.(rand(Uniform(0, 1), B), digits=4))
sdX = 2 * muX
paramsX = (muX, sdX)

distrTypeY = LogNormal{dtype}
muY = dtype.(round.(rand(Uniform(0, 1), B), digits=4))
sdY = 2 * muY
paramsY = (muY, sdY)


@time main(B, S, nx, ny, distrTypeX, paramsX, distrTypeY, paramsY, dtype=dtype, seed=seed)

(x[:, 1, 1])[1] = 0.8590659f0
